# Sentiment Analyst -  Naive Bayes

## 1. Introduction:

Dans ce cahier, nous utiliserons le framework Spark pour construire un modèle d'analyse des sentiments basé sur l'algorithme Naive Bayes.

L'ensemble de données est un avis de plusieurs utilisateurs sur le bot Amazon Alexa, où chaque ligne a un `review` et un `rating` (de 1 à 5 étoiles) et d'autres colonnes.  
Lien pour télécharger les données: [Data Source](https://www.kaggle.com/sid321axn/amazon-alexa-reviews#amazon_alexa.tsv)

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, udf
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, \
    IDF, StopWordsRemover, StringIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import IntegerType

import re

In [3]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Sentiment Analysis") \
    .getOrCreate()

## 1. Chargement des données

In [4]:
path = 'data/amazon-alexa-reviews.tsv'

In [5]:
df = spark.read.option("sep", "\t") \
    .option("header", "true") \
    .csv(path)

In [6]:
df.show(3)

+------+---------+----------------+--------------------+--------+
|rating|     date|       variation|    verified_reviews|feedback|
+------+---------+----------------+--------------------+--------+
|     5|31-Jul-18|Charcoal Fabric |       Love my Echo!|       1|
|     5|31-Jul-18|Charcoal Fabric |           Loved it!|       1|
|     4|31-Jul-18|  Walnut Finish |Sometimes while p...|       1|
+------+---------+----------------+--------------------+--------+
only showing top 3 rows



Ce qui nous intéresse sont les deux colonnes `rating` et `verified_reviews`.  Donc nous allons vérifier si ces deux colonnes contient déja des valeurs null.

In [7]:
df.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rating            3150 non-null   object
 1   date              3150 non-null   object
 2   variation         3150 non-null   object
 3   verified_reviews  3150 non-null   object
 4   feedback          3150 non-null   object
dtypes: object(5)
memory usage: 123.2+ KB


Perfecto! aucune valeur null.  
Pour comparer le nombre des `rating`:

In [8]:
df.groupBy('rating').agg(count('rating')).orderBy('rating').show()

+------+-------------+
|rating|count(rating)|
+------+-------------+
|     1|          161|
|     2|           96|
|     3|          152|
|     4|          455|
|     5|         2286|
+------+-------------+



On remarque que les rating du 5 étoiles domine le dataset.  
Maintenant nous pouvons nous débarrasser des colonnes inutiles:

Maintenant nous allons créer les 2 classes des sentiment:
0. => Sentiment Négatif (1-3 étoiles)
1. => Sentiment Positif (4-5 étoiles)

In [9]:
label_col = udf(lambda x: int((x =='5')|(x=='4')), IntegerType())  
df = df.withColumn('classe', label_col(df.rating))
df.show(1)

+------+---------+----------------+----------------+--------+------+
|rating|     date|       variation|verified_reviews|feedback|classe|
+------+---------+----------------+----------------+--------+------+
|     5|31-Jul-18|Charcoal Fabric |   Love my Echo!|       1|     1|
+------+---------+----------------+----------------+--------+------+
only showing top 1 row



In [10]:
dfn = df.drop(*['date', 'variation', 'feedback'])

In [11]:
dfn.show(3)

+------+--------------------+------+
|rating|    verified_reviews|classe|
+------+--------------------+------+
|     5|       Love my Echo!|     1|
|     5|           Loved it!|     1|
|     4|Sometimes while p...|     1|
+------+--------------------+------+
only showing top 3 rows



## 3. Extraction des features

Dans cette partie, nous allons extraire les features nécessaires pour l'apprentisage de notre model à partir de la colonne `verified_reviews`.  
Mais avant ça nous allons préparer nos données, en supprimant les tags HTML, les mots vides (et, dans ..), déplacer les emojis à la fin du texte, et transformer le texte en miniscule.

In [12]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', str(text))
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [13]:
preprocessor_udf = udf(preprocessor)

In [14]:
dfn = dfn.withColumn('prepared_reviews', preprocessor_udf(col('verified_reviews')))

In [15]:
dfn.show(3)

+------+--------------------+------+--------------------+
|rating|    verified_reviews|classe|    prepared_reviews|
+------+--------------------+------+--------------------+
|     5|       Love my Echo!|     1|       love my echo |
|     5|           Loved it!|     1|           loved it |
|     4|Sometimes while p...|     1|sometimes while p...|
+------+--------------------+------+--------------------+
only showing top 3 rows



In [16]:
# diviser le texte en mots séparés
regexTokenizer = RegexTokenizer(inputCol="prepared_reviews", outputCol="mots", pattern="\\W")
dfn = regexTokenizer.transform(dfn)

In [17]:
dfn.show(1)

+------+----------------+------+----------------+----------------+
|rating|verified_reviews|classe|prepared_reviews|            mots|
+------+----------------+------+----------------+----------------+
|     5|   Love my Echo!|     1|   love my echo |[love, my, echo]|
+------+----------------+------+----------------+----------------+
only showing top 1 row



In [18]:
# supprimer les mots vides
remover = StopWordsRemover(inputCol='mots', outputCol='mots_clean')
dfn = remover.transform(dfn).select('rating', 'mots_clean')

In [19]:
dfn.show(2)

+------+------------+
|rating|  mots_clean|
+------+------------+
|     5|[love, echo]|
|     5|     [loved]|
+------+------------+
only showing top 2 rows



In [20]:
# trouver le terme fréquences des mots
cv = CountVectorizer(inputCol="mots_clean", outputCol="TF")
cvmodel = cv.fit(dfn)
dfn = cvmodel.transform(dfn)
dfn.take(1)

[Row(rating='5', mots_clean=['love', 'echo'], TF=SparseVector(3947, {0: 1.0, 1: 1.0}))]

In [28]:
# trouver le Inter-document Frequency
idf = IDF(inputCol="TF", outputCol="features")
idfModel = idf.fit(dfn)
dfn = idfModel.transform(dfn)
dfn.head()

Row(rating='5', mots_clean=['love', 'echo'], TF=SparseVector(3947, {0: 1.0, 1: 1.0}), TFIDF=SparseVector(3947, {0: 1.334, 1: 1.6652}), features=SparseVector(3947, {0: 1.334, 1: 1.6652}))

In [22]:
# créer la colonne d'étiquette
indexer = StringIndexer(inputCol="classe", outputCol="label")

## 4. Créer le ML Pipeline

Aprés définir tout les fonctions nécaissaires pour traiter notre dataset, nous allons les enchainer dans un pipeline pour construire notre modele.

In [23]:
data = df.drop(*['date', 'variation', 'feedback'])
data = data.withColumn('prepared_reviews', preprocessor_udf(col('verified_reviews')))

In [25]:
# Divisez le jeu de données au hasard en ensembles de formation et de test
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed = 100)

In [29]:
# créer le pipeline
nb = NaiveBayes(smoothing=1.0)
pipeline = Pipeline(stages=[regexTokenizer, remover, cv, idf, indexer, nb])

In [30]:
# éxecuter les étapes du pipeline et former le modele
model = pipeline.fit(trainingData)

In [31]:
# Faire des prédictions sur testData 
#afin que nous puissions mesurer la précision de notre modèle sur de nouvelles données
predictions = model.transform(testData)

Pour évaluer notre modele nous allons utiliser `Evaluator` de `MulticlassClassification`.

In [32]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",\
            metricName="accuracy")

In [33]:
accuracy = evaluator.evaluate(predictions)
print("Model Accuracy: ", accuracy)

Model Accuracy:  0.8923240938166311


Bon, une précision de `90%` sur ce simple dataset n'est pas mauvais. 

In [34]:
spark.stop()